In [ ]:
import requests
import json
import time
from datetime import datetime, timedelta
from itertools import permutations
from itertools import combinations

import networkx as nx
from networkx.classes.function import path_weight

import matplotlib.pyplot as plt

#information that we are using to find the currencies
ids = ['ripple', 'bitcoin-cash', 'eos', 'litecoin', 'ethereum', 'bitcoin', 'cardano']
vs_currencies = ['xrp', 'bch', 'eos', 'ltc', 'eth', 'btc', 'ada']

url = "https://api.coingecko.com/api/v3/simple/price?ids=ripple%2Ccardano%2Cbitcoin-cash%2Ceos%2Clitecoin%2Cethereum%2Cbitcoin&vs_currencies=xrp%2Cada%2Cbch%2Ceos%2Cltc%2Ceth%2Cbtc"

#creates the json from the url above
req = requests.get(url)
dct1 = json.loads(req.text)

#print(dct1)

edges = []


#creates the edges
j = 0
for c1 in ids:
    i = 0
    for c2 in vs_currencies:
        if j != i:
            try:
                edges.append((vs_currencies[j], vs_currencies[i], dct1[c1][c2]))
            except:
                pass
        i += 1
    j += 1

#print(edges)

#creates the graphs
g = nx.DiGraph()
g.add_weighted_edges_from(edges) #relationship is directional
pos=nx.circular_layout(g)
nx.draw_networkx(g,pos)
labels = nx.get_edge_attributes(g,'weight')
nx.draw_networkx_edge_labels(g,pos,edge_labels=labels)
plt.savefig("graph.png")

#sets variables 
min_weight_factor = 10000
max_weight_factor = 0
min_path = 0
max_path = 0
min_path_reversed = 0
max_path_reversed = 0
usedstartandend = []

#creates the paths
for n1, n2 in permutations(g.nodes,2): #permutations returns all pairs
    #print("paths from",n1,"to",n2,"----------------")
    for path in nx.all_simple_paths(g, source=n1, target=n2):
        #print(path)
        
        path_weight = 1
        reverse_weight = 1
        #creates the weight for the original paths
        for i in range(len(path) - 1):
            path_weight *= g[path[i]][path[i+1]]['weight']
            
        path_reversed = path[::-1] #reverses the path
        #creates the weight for the reversed paths 
        for i in range(len(path_reversed) - 1):
            try:
                reverse_weight *= g[path_reversed[i]][path_reversed[i+1]]['weight']
               
            except:
                reverse_weight = 0
                
        
        weight_factor = reverse_weight * path_weight #calculates the weight factor for both directions of the path  
            
        # usedstartandend = []    
        #finds the paths that have a value and prints
        if weight_factor != 0:
            if (n1,n2) not in usedstartandend:
                print("\n----------","paths from", n1,"to", n2,"---------------")
            usedstartandend.append((n1,n2))
            
            print("\nPath: ", path, "Reversed Path: ", path_reversed)
            print("Path Weight: ", path_weight, "\t", "Reversed Weight: ", reverse_weight,"\t")
            print("Weight Factor: ", weight_factor)
            #finds the minimum weight factor and both of its paths
            if weight_factor < min_weight_factor:
                min_weight_factor = weight_factor
                min_path = path
                min_path_reversed = path_reversed
            #finds the maxiumum weight factor and both of its paths    
            if weight_factor > max_weight_factor:
                max_weight_factor = weight_factor
                max_path = path
                max_path_reversed = path_reversed
            
# print("Max Weight Factor: ", max_weight_factor)
# print("Max Path:", max_path, "Max Reversed Path: ", max_path_reversed)
# print('Min Weight Factor: ', min_weight_factor)
# print("Min Path: ", min_path, "Min Reseversed Path: ", min_path_reversed)
# input("keep input alive")

print("\n")
print("--------Results--------")
print("Highest Weight Factor","\t", "Path and Reversed Path")
print(max_weight_factor,"\t", max_path, max_path_reversed)
print("Lowest Weight Factor","\t", "Path and Reversed Path")
print(min_weight_factor,"\t", min_path, min_path_reversed)
input("keep input alive")